In [1]:
import org.locationtech.jts.geom._
import org.apache.spark.sql.types._
import org.locationtech.geomesa.spark.jts._

// import spark.implicits._
// below hack
val spark2: SparkSession = spark
import spark2.implicits._
spark.withJTS

import org.apache.spark.graphx._
// To make some of the examples work we will also need RDD
import org.apache.spark.rdd.RDD

spark2 = org.apache.spark.sql.SparkSession@f7b54e7


org.apache.spark.sql.SparkSession@f7b54e7

In [2]:
val MAX_VERTICES = 1000

MAX_VERTICES = 1000


1000

In [3]:
val edges = spark2.read.load("file:///geodata/ffe_%1$s_edges.parquet" format (MAX_VERTICES))
edges.cache()
edges.count()

edges = [id: int, near_id: int]


3966

In [4]:
// vertices.rdd.first()
val edge_rdd: RDD[Edge[Any]] = 
    edges.rdd.map(x => Edge(x.getInt(0), x.getInt(1), null))
edge_rdd.first()                       

edge_rdd = MapPartitionsRDD[17] at map at <console>:47


Edge(959,961,null)

In [5]:
val vertices = spark2.read.load("file:///geodata/ffe_%1$s_vertices.parquet" format (MAX_VERTICES))
vertices.cache()
vertices.count()

vertices = [_c0: int, TARGET_FID: int ... 15 more fields]


1000

In [6]:
// vertices.rdd.first()
val vert_rdd: RDD[(VertexId, (Int, String, Any, Any))] = 
    vertices.rdd.map(x => (x.getInt(0), (x.getInt(1), x.getString(2), x.get(9), null)))
vert_rdd.first()

vert_rdd = MapPartitionsRDD[36] at map at <console>:47


(0,(1,Houghton Bay,POLYGON ((1749551.504199982 5422104.252850056, 1749551.544950008 5422110.366449833, 1749556.703549862 5422110.332049847, 1749556.662749767 5422104.218400002, 1749551.504199982 5422104.252850056)),null))

In [7]:
// Build the initial Graph
val graph = Graph(vert_rdd, edge_rdd)

graph = org.apache.spark.graphx.impl.GraphImpl@1ec7fc0e


org.apache.spark.graphx.impl.GraphImpl@1ec7fc0e

In [8]:
val facts: RDD[String] =
  graph.triplets.map(triplet =>
    triplet.srcAttr._1 + " is the " + triplet.attr + " of " + triplet.dstAttr._1)
facts.take(50)

facts = MapPartitionsRDD[54] at map at <console>:46


Array(28 is the null of 31, 74 is the null of 76, 210 is the null of 209, 223 is the null of 224, 248 is the null of 249, 248 is the null of 250, 320 is the null of 322, 404 is the null of 408, 445 is the null of 444, 535 is the null of 543, 550 is the null of 716, 550 is the null of 718, 550 is the null of 719, 550 is the null of 721, 550 is the null of 722, 550 is the null of 723, 550 is the null of 724, 550 is the null of 725, 550 is the null of 726, 550 is the null of 727, 550 is the null of 728, 550 is the null of 732, 550 is the null of 739, 550 is the null of 740, 550 is the null of 741, 551 is the null of 552, 551 is the null of 988, 551 is the null of 989, 551 is the nul...

In [9]:
graph.numEdges

3966

In [10]:
graph.inDegrees.first()

(772,4)

In [11]:
graph.triangleCount().vertices.take(10)

Array((451,0), (454,0), (147,0), (155,0), (772,5), (752,64), (586,41), (667,3), (428,0), (464,0))

## WIP playing with the API

In [12]:
val sourceId: VertexId = graph.vertices.take(10)(9)._1 // just some random vertex
// Initialize the graph such that all vertices except the root have distance infinity.
val initialGraph = graph.mapVertices((id, _) =>
    if (id == sourceId) 0.0 else Double.PositiveInfinity)

sourceId = 464
initialGraph = org.apache.spark.graphx.impl.GraphImpl@6dc231f0


org.apache.spark.graphx.impl.GraphImpl@6dc231f0

In [13]:
graph.vertices.take(10)(9)._1

464

In [21]:
val facts: RDD[String] =
  initialGraph.triplets.map(triplet =>
    triplet.srcId + ":" + triplet.srcAttr + " --> " + triplet.dstId + ":" + triplet.dstAttr)
facts.take(50)

facts = MapPartitionsRDD[120] at map at <console>:48


Array(26:Infinity --> 29:Infinity, 72:Infinity --> 74:Infinity, 202:Infinity --> 201:Infinity, 215:Infinity --> 216:Infinity, 240:Infinity --> 241:Infinity, 240:Infinity --> 242:Infinity, 312:Infinity --> 314:Infinity, 395:Infinity --> 399:Infinity, 434:Infinity --> 433:Infinity, 524:Infinity --> 532:Infinity, 538:Infinity --> 704:Infinity, 538:Infinity --> 706:Infinity, 538:Infinity --> 707:Infinity, 538:Infinity --> 709:Infinity, 538:Infinity --> 710:Infinity, 538:Infinity --> 711:Infinity, 538:Infinity --> 712:Infinity, 538:Infinity --> 713:Infinity, 538:Infinity --> 714:Infinity, 538:Infinity --> 715:Infinity, 538:Infinity --> 716:Infinity, 538:Infinity --> 720:Infinity, 538...